In [ ]:
import random
import json
import pandas as pd

#global variables to be manipulated
stimulus_set = []
relay = 0

#defining stimulus set
vertical_con_list = [
    {"prime":"le", "probe":"le", "congruency":"congruent", "correct_response":"n", "name":"vertical_con_1"},
    {"prime":"fel", "probe":"fel", "congruency":"congruent", "correct_response":"j", "name":"vertical_con_2"}
]
vertical_inc_list = [
    {"prime":"fel", "probe":"le", "congruency":"incongruent", "correct_response":"n", "name":"vertical_incon_1"},
    {"prime":"le", "probe":"fel", "congruency":"incongruent", "correct_response":"j", "name":"vertical_incon_2"}
    
]

horizontal_con_list = [
    {"prime": "bal", "probe": "bal", "congruency":"congruent", "correct_response":"f", "name": "horizontal_con_1"},
    {"prime": "jobb", "probe": "jobb", "congruency":"congruent", "correct_response":"g", "name": "horizontal_con_2"}
    ]
horizontal_incon_list = [
    {"prime": "bal", "probe": "jobb", "congruency":"incongruent", "correct_response":"g", "name": "horizontal_incon_1"},
    {"prime": "jobb", "probe": "bal", "congruency":"incongruent", "correct_response":"f", "name":"horizontal_incon_2"}
]

#grouping congruent and incongruent stimuli together
vertical_group = [vertical_con_list, vertical_inc_list]
horizontal_group = [horizontal_con_list, horizontal_incon_list]

def stimulus_finder(group):
    global stimulus_set
    congruency = random.randint(0,len(group)-1)
    stimulus = random.randint(0,len(group[congruency])-1)
    stimulus_set.append(group[congruency][stimulus])

def group_relay():
    global relay
    if relay == 0:
        stimulus_finder(horizontal_group)
        relay = 1
    else:
        stimulus_finder(vertical_group)
        relay = 0

def set_factory(times):
    for i in range(times):
        group_relay()
trial_rate = []
trial_count = 80
def factor():
    global trial_rate
    global trial_count
    set_factory(trial_count)
    con_count = 0
    incon_count = 0
    for i in stimulus_set:
        if i["congruency"] == "congruent":
            con_count += 1
        elif i["congruency"] == "incongruent":
            incon_count += 1
    con = con_count/trial_count
    incon = incon_count/trial_count
    trial_rate = [con, incon]

match_count = 0
while True:
    if trial_rate==[0.5, 0.5]:
        trial_block = pd.DataFrame(stimulus_set, columns=["prime", "probe", "congruency", "correct_response"])
        trial_block["previous_congruency"] = trial_block["congruency"].shift(1)
        trial_block["con_pair"] = trial_block["previous_congruency"] + "-" + trial_block["congruency"]
        pair_counts = trial_block ["con_pair"].value_counts()
        if pair_counts["congruent-congruent"] == 20 and pair_counts["congruent-incongruent"] == 20 and pair_counts["incongruent-congruent"] == 20 and pair_counts["incongruent-incongruent"] == 20:
            match_count += 1
            print("Match found: {}".format(match_count))
            print("Correct paircount found, writing...")
            with open("trials.json", "r+") as file:
                #First we load existing data into a dictionary
                file_data = json.load(file)
                #join new_data with file_data inside emp_details
                file_data["trials"].append(stimulus_set)
                #Sets file's current position at offset.
                file.seek(0)
                #convert back to json.
                json.dump(file_data, file)
            
    stimulus_set = []
    factor()

KeyboardInterrupt: 